In [1]:
from variable.variable import Variable
from activation_functions.relu import ReLUActivation
relu_fn = ReLUActivation()

In [2]:
x = Variable(3)
y = Variable(4)

In [3]:
z = x + y 

In [4]:
z

Variable(data=7, grad=0), _op=<ops.basic_ops.AddOperation object at 0x0000019394B446E0>

In [5]:
z_relu = relu_fn(z)

In [6]:
z_relu.backward()

In [7]:
x, y

(Variable(data=3, grad=1), Variable(data=4, grad=1))

In [1]:
from nn.linear_layer import LinearLayer
from variable.variable import Variable

x = LinearLayer(5, 10)

In [2]:
# we do batch size of 3 and input size of 5

input_data = [[Variable(i) for i in range(5)] for _ in range(3)]

In [3]:
x(input_data)[0][0]

Variable(data=2.409617244364817, grad=0), _op=<ops.basic_ops.AddOperation object at 0x0000024420CC3E90>, history={Variable(data=0.47074814348000604, grad=0), _op=None, history=set(), Variable(data=1.9388691008848113, grad=0), _op=<ops.basic_ops.AddOperation object at 0x0000024420CC3DA0>, history={Variable(data=0.23801442405694906, grad=0), _op=None, history=set(), Variable(data=1.7008546768278623, grad=0), _op=<ops.basic_ops.AddOperation object at 0x0000024420CC3CB0>, history={Variable(data=0.028448334035315992, grad=0), _op=None, history=set(), Variable(data=1.6724063427925464, grad=0), _op=<ops.basic_ops.AddOperation object at 0x0000024420CC3BC0>, history={Variable(data=0.9948622463707882, grad=0), _op=None, history=set(), Variable(data=0.6775440964217583, grad=0), _op=<ops.basic_ops.AddOperation object at 0x0000024420CC3B60>, history={Variable(data=0, grad=0), _op=None, history=set(), Variable(data=0.6775440964217583, grad=0), _op=None, history=set()}}}}}